# Accuaring County the coordinates data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## we need the geopandas and geopy to get coordinates of the of the County in the data set 
## Geocoding service is built on top of OpenStreetMap data
!pip install geopandas
!pip install geopy


     |████████████████████████████████| 1.0MB 6.0MB/s 
     |████████████████████████████████| 6.5MB 8.3MB/s 
     |████████████████████████████████| 15.3MB 245kB/s 


### Importing the data


In [3]:
root_URL = 'https://raw.githubusercontent.com/StevenVuong/WDL_2020/master/data/stage1'
Churn_OD_file = 'Churn_OD.txt'
Churn_UsersProfile_file = 'Churn_UsersProfile.txt'

Churn_OD_URL = root_URL +'/'+ Churn_OD_file
Churn_UsersProfile_URL = root_URL +'/'+ Churn_UsersProfile_file


In [4]:
df1 = pd.read_csv(Churn_OD_URL ,delimiter="|",encoding = 'Latin1')


In [5]:
df1.head(10)

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight
0,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110608,0.307323
1,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110639,0.069997
2,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110658,0.066059
3,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110654,0.059847
4,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110633,0.052341
5,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110611,0.049032
6,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,AMADORA,111512,0.039779
7,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110660,0.038691
8,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110610,0.037827
9,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110666,0.037800


In [22]:
!pip install fsspec
!pip install aiohttp

     |████████████████████████████████| 1.3MB 5.7MB/s 
     |████████████████████████████████| 143kB 23.7MB/s 
     |████████████████████████████████| 296kB 21.0MB/s 


In [23]:
from subprocess import check_output
import geopandas
import fsspec
import aiohttp
# https://github.com/Hardi-Lior/WDL/blob/main/Stage_1/data/Cont_AAD_CAOP2020/Cont_AAD_CAOP2020.shp.zip

In [26]:
path = "simplecache::https://github.com/Hardi-Lior/WDL/blob/main/Stage_1/data/Cont_AAD_CAOP2020/Cont_AAD_CAOP2020.shp.zip"
with fsspec.open(path) as file:
    df = geopandas.read_file(file)



# zipfile ='https://github.com/Hardi-Lior/WDL/blob/main/Stage_1/data/Cont_AAD_CAOP2020/Cont_AAD_CAOP2020.shp.zip'
# states = geopandas.read_file(zipfile)

DriverError: ignored